In [1]:
import os
import gc  #This is garbage collector 
import sys #System 
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline
sns.set_style('darkgrid')
pd.set_option('display.max_columns', 140)

from sklearn.preprocessing import StandardScaler as scale
from sklearn.decomposition import PCA
from sklearn.cluster import k_means

In [2]:
#Reading inputs 
with open ('Byproducts/df_pre_pca', 'rb') as fp:
    df_pre_pca = pickle.load(fp)

with open ('Byproducts/pca_object', 'rb') as fp:
    pca = pickle.load(fp)
    
with open ('Byproducts/high_null_list', 'rb') as fp:
    high_null_list = pickle.load(fp)
    
with open ('Byproducts/scalar_object', 'rb') as fp:
    scaler = pickle.load(fp)
    
n_components = 30 
pc_names = ['PC'+str(i) for i in range(n_components)]

with open ('Byproducts/model1_xgboost', 'rb') as fp:
    model1 = pickle.load(fp)
    
with open ('Byproducts/model3_xgboost', 'rb') as fp:
    model3 = pickle.load(fp)


In [3]:
import janestreet

env = janestreet.make_env()
iter_test = env.iter_test()


In [4]:
cnt = 0 
for (test_df, sample_prediction_df) in iter_test:
    X_test = test_df.loc[:, test_df.columns.str.contains('feature')]

    #preprocess X_test
    X_test = X_test.drop(columns = high_null_list, axis=1)
    X_test.fillna(df_pre_pca.mean(), inplace = True)
    X_test = scaler.transform(X_test[X_test.columns.values])
    X_test = pd.DataFrame(pca.transform(X_test), columns = pc_names  )


    y_preds = model1.predict(X_test) + model3.predict(X_test)
    if y_preds == 2:
        y_preds = np.array([1])
    else:
        y_preds = np.array([0])

    sample_prediction_df.action = y_preds
    env.predict(sample_prediction_df)
    cnt = cnt + 1


In [5]:
env.predictions

[   action
 0       1,
    action
 1       0,
    action
 2       1,
    action
 3       1,
    action
 4       0]